# Preprocessing
This notebook contains all the steps we performed for data preprocessing.

### Import Packages

In [1]:
from faker import Faker
from lxml import etree
import numpy as np
import pandas as pd
import pickle
import string
import time
import urllib
pd.set_option('display.max_columns', None)

### Import Data

In [2]:
df_books = pd.read_json("goodreads_books_poetry.json", lines=True)
df_interactions = pd.read_json("goodreads_interactions_poetry.json", lines=True)
df_reviews = pd.read_json("goodreads_reviews_poetry.json", lines=True)
df_authors = pd.read_json("goodreads_book_authors.json", lines=True)
df_series = pd.read_json("goodreads_book_series.json", lines=True)

### Web Scraping
The `df_books` dataframe contains an `image_url` column with links to the book cover images. On manual inspection we noticed that several of the links were broken, and that the broken links were those that contained the substring "nophoto" in the URL. As shown below, 15861 links were broken, which is ~43% of the size of the dataset.

In [4]:
print(f"No. of broken links: {sum(df_books.image_url.str.contains('nophoto'))}")
print(f"No. of broken links (as a % of the no. of books): {(sum(df_books.image_url.str.contains('nophoto')) / len(df_books) * 100):.2f}")

No. of broken links: 15861
No. of broken links (as a % of the no. of books): 43.44


We believe that displaying the images adds substantial value to the aesthetics of the web application. Hence we wrote a script that replaces the broken links with links to the book cover images scraped from Goodreads. This was possible as `df_books` also contains a `link` column that links to the Goodreads page of the book. Our script retrieves the Goodreads page HTMLs and parses the book cover image links using xpath.

The script is shown below.

In [ ]:
# Cache to store links to book cover images
image_urls = [None for _ in range(len(df_books))]
# No. of attempts for each book. This is needed as sometimes the request fails
# to retrieve the HTML page due to network issues
max_tries = 3
# Running count of no. of books with unbroken/successfully scraped links
success_count = 0
# Running count of no. of books with unsuccessfully scraped links 
failure_count = 0
# No. of iterations after which to save the image URLs to disk
checkpoint_every = 1000

start_time = time.time()

for i, row in df_books[["link", "image_url"]].iterrows():
    if "nophoto" not in row["image_url"]: # Link is unbroken
        image_urls[i] = row["image_url"]
    else:
        tries = 0

        # Retrieve the Goodreads page HTML and scrape the book cover image link
        # using xpath
        while image_urls[i] is None and tries < max_tries:
            try:
                response = urllib.request.urlopen(row["link"])
                tree = etree.HTML(response.read().decode("utf-8"))
                image_url = tree.xpath("//img[@class='ResponsiveImage' and @role='presentation']/@src")[0]
                image_urls[i] = image_url
            except:
                pass

            tries += 1

    if image_urls[i] is not None: # Link is now unbroken
        success_count += 1
    else: # Link is still broken
        failure_count += 1
    
    if (i + 1) % checkpoint_every == 0: # Save checkpoint
        with open(f"checkpoint_{i + 1}.pickle", mode="wb") as f:
            pickle.dump(image_urls, f)
    
    if i % 100 == 0: # Print logs
        print(f"Completed for link at index {i} | #success: {success_count}, #failures: {failure_count}, elapsed: {((time.time() - start_time) / 60):.2f} mins")

with open(f"checkpoint_final.pickle", mode="wb") as f:
    pickle.dump(image_urls, f)

After running the script only 2 of the links remained broken. 

Also, we decided not to download each of the images as that would have taken a substantial amount of time. Instead, to display the images our application simply fetches the images from the links at runtime.

### Creating `Book` Dataset

In [ ]:
# Load book cover image URLs
with open(f"checkpoint_final.pickle", mode="rb") as f:
    image_urls_ = pickle.load(f)
df_books["image_url"] = pd.Series(image_urls_)

# Create copy of dataframe
df_books_cleaned = df_books.copy()

# Print column data types
print(df_books_cleaned.dtypes)
print("")

# Keep neccessary columns
df_books_cleaned = df_books_cleaned[["book_id", "title", "description", "language_code", "edition_information", "format", "is_ebook", "isbn", "isbn13", "asin", "kindle_asin", "publisher", "num_pages", "image_url"]]

# Create publish_date column
df_books_cleaned["publish_date"] = pd.to_datetime(df_books.apply(lambda row: f"{row['publication_year']}-{row['publication_month']}-{row['publication_day']}" if all([row['publication_year'] != "", row['publication_month'] != "", row['publication_day'] != "", (row['publication_year'] != "" and int(row['publication_year']) <= 2022)]) else "", axis=1), infer_datetime_format=True, errors="coerce")

# Check book_id range
print(df_books_cleaned["book_id"].min())
print(df_books_cleaned["book_id"].max())
print("")

# Clean title column
df_books_cleaned["title"] = df_books_cleaned["title"].apply(lambda s: s.strip(" '"))
df_books_cleaned["title"] = df_books_cleaned["title"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["title"].apply(lambda s: len(s)).min())
print(df_books_cleaned["title"].apply(lambda s: len(s)).max())
df_books_cleaned["title"] = df_books_cleaned["title"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["title"] = df_books_cleaned["title"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["title"] = df_books_cleaned["title"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["title"].apply(lambda s: len(s) == 0)))
# Remove books with no title
df_books_cleaned = df_books_cleaned[df_books_cleaned["title"].apply(lambda s: len(s) != 0)]
print("")

# Clean description column
df_books_cleaned["description"] = df_books_cleaned["description"].apply(lambda s: s.strip(" '"))
df_books_cleaned["description"] = df_books_cleaned["description"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["description"].apply(lambda s: len(s)).min())
print(df_books_cleaned["description"].apply(lambda s: len(s)).max())
df_books_cleaned["description"] = df_books_cleaned["description"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["description"] = df_books_cleaned["description"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["description"] = df_books_cleaned["description"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["description"].apply(lambda s: len(s) == 0)))
print("")

# Clean language_code column
df_books_cleaned["language_code"] = df_books_cleaned["language_code"].apply(lambda s: s.strip(" '"))
df_books_cleaned["language_code"] = df_books_cleaned["language_code"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["language_code"].apply(lambda s: len(s)).min())
print(df_books_cleaned["language_code"].apply(lambda s: len(s)).max())
df_books_cleaned["language_code"] = df_books_cleaned["language_code"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["language_code"] = df_books_cleaned["language_code"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["language_code"] = df_books_cleaned["language_code"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["language_code"].apply(lambda s: len(s) == 0)))
print("")

# Clean edition_information column
df_books_cleaned["edition_information"] = df_books_cleaned["edition_information"].apply(lambda s: s.strip(" '"))
df_books_cleaned["edition_information"] = df_books_cleaned["edition_information"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["edition_information"].apply(lambda s: len(s)).min())
print(df_books_cleaned["edition_information"].apply(lambda s: len(s)).max())
df_books_cleaned["edition_information"] = df_books_cleaned["edition_information"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["edition_information"] = df_books_cleaned["edition_information"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["edition_information"] = df_books_cleaned["edition_information"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["edition_information"].apply(lambda s: len(s) == 0)))
print("")

# Clean format column
df_books_cleaned["format"] = df_books_cleaned["format"].apply(lambda s: s.strip(" '"))
df_books_cleaned["format"] = df_books_cleaned["format"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["format"].apply(lambda s: len(s)).min())
print(df_books_cleaned["format"].apply(lambda s: len(s)).max())
df_books_cleaned["format"] = df_books_cleaned["format"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["format"] = df_books_cleaned["format"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["format"] = df_books_cleaned["format"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["format"].apply(lambda s: len(s) == 0)))
print("")

# Clean is_ebook column
print(set(df_books_cleaned["is_ebook"]))
df_books_cleaned["is_ebook"] = df_books_cleaned["is_ebook"].apply(lambda s: True if s == "true" else False)
print("")

def is_valid_code(code: str, req_len: int, valid_chars: set[str]) -> bool:
    return len(code) == req_len and all(c in valid_chars for c in code)
# Clean isbn column
valid_chars: set[str] = set(string.digits + "X")
df_books_cleaned["isbn"] = df_books_cleaned["isbn"].apply(
    lambda code: str(code) if is_valid_code(code, 10, valid_chars) else ""
)
# Clean isbn13 column
df_books_cleaned["isbn13"] = df_books_cleaned["isbn13"].apply(
    lambda code: str(code) if is_valid_code(code, 13, valid_chars) else ""
)
# Clean asin column
valid_chars: set[str] = set(string.ascii_uppercase + string.digits)
df_books_cleaned["asin"] = df_books_cleaned["asin"].apply(
    lambda code: str(code) if is_valid_code(code, 10, valid_chars) else ""
)
# Clean kindle_asin column
df_books_cleaned["kindle_asin"] = df_books_cleaned["kindle_asin"].apply(
    lambda code: str(code) if is_valid_code(code, 10, valid_chars) else ""
)
print(sum(df_books_cleaned["isbn"].apply(lambda s: s == "")))
print(sum(df_books_cleaned["isbn13"].apply(lambda s: s == "")))
print(sum(df_books_cleaned["asin"].apply(lambda s: s == "")))
print(sum(df_books_cleaned["kindle_asin"].apply(lambda s: s == "")))
print("")

# Clean publisher column
df_books_cleaned["publisher"] = df_books_cleaned["publisher"].apply(lambda s: s.strip(" '"))
df_books_cleaned["publisher"] = df_books_cleaned["publisher"].apply(lambda s: s.strip('"'))
print(df_books_cleaned["publisher"].apply(lambda s: len(s)).min())
print(df_books_cleaned["publisher"].apply(lambda s: len(s)).max())
df_books_cleaned["publisher"] = df_books_cleaned["publisher"].apply(lambda s: s.replace("\n", ""))
df_books_cleaned["publisher"] = df_books_cleaned["publisher"].apply(lambda s: s.replace("\r", ""))
df_books_cleaned["publisher"] = df_books_cleaned["publisher"].apply(lambda s: s.replace("\t", ""))
print(sum(df_books_cleaned["publisher"].apply(lambda s: len(s) == 0)))
print("")

# Clean num_pages column
df_books_cleaned["num_pages"] = df_books_cleaned["num_pages"].apply(lambda s: int(s) if (s != "" and int(s) > 0) else None)

# Rename columns
df_books_cleaned = df_books_cleaned.rename(columns={"book_id": "id", "edition_information": "edition"})

# Save to disk
df_books_cleaned.to_csv("book.csv", sep="\t", index=False)

### Creating `Similar_Books` Dataset

In [ ]:
valid_book_ids = set(df_books_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)][["book_id", "similar_books"]]

d = {"book_id1": [], "book_id2": []}

for i, row in df_books_.iterrows():
    for similar_book in row["similar_books"]:
        if int(similar_book) in valid_book_ids:
            d["book_id1"].append(row["book_id"])
            d["book_id2"].append(int(similar_book))
            d["book_id2"].append(row["book_id"])
            d["book_id1"].append(int(similar_book))

# Create dataframe
similar_books = pd.DataFrame.from_dict(d)

# Drop duplicate rows
similar_books = similar_books.drop_duplicates()

# Print column data types
print(similar_books.dtypes)

# Save to disk
similar_books.to_csv("similar_books.csv", sep="\t", index=False)

### Creating `Series` Dataset

In [ ]:
# Create copy of dataframe
df_series_cleaned = df_series.copy()

# Keep neccessary columns
df_series_cleaned = df_series_cleaned[["series_id", "title", "description", "numbered"]]

# Print column data types
print(df_series_cleaned.dtypes)
print("")

# Clean series_id column
print(len(df_series_cleaned))
print(df_series_cleaned["series_id"].nunique())
print(df_series_cleaned["series_id"].min())
print(df_series_cleaned["series_id"].max())
valid_book_ids = set(df_books_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)]
valid_series_ids = set(int(series_id) for ls in df_books_["series"] for series_id in ls)
df_series_cleaned = df_series_cleaned[df_series_cleaned["series_id"].apply(lambda x: x in valid_series_ids)]
print("")

# Clean title column
df_series_cleaned["title"] = df_series_cleaned["title"].apply(lambda s: s.strip(" '"))
df_series_cleaned["title"] = df_series_cleaned["title"].apply(lambda s: s.strip('"'))
print(df_series_cleaned["title"].apply(lambda s: len(s)).max())
df_series_cleaned["title"] = df_series_cleaned["title"].apply(lambda s: s.replace("\n", ""))
df_series_cleaned["title"] = df_series_cleaned["title"].apply(lambda s: s.replace("\r", ""))
df_series_cleaned["title"] = df_series_cleaned["title"].apply(lambda s: s.replace("\t", ""))
print(sum(df_series_cleaned["title"].apply(lambda s: len(s) == 0)))
# Remove series with no title
df_series_cleaned = df_series_cleaned[df_series_cleaned["title"].apply(lambda s: len(s) != 0)]
print("")

# Clean description column
df_series_cleaned["description"] = df_series_cleaned["description"].apply(lambda s: s.strip(" '"))
df_series_cleaned["description"] = df_series_cleaned["description"].apply(lambda s: s.strip('"'))
print(df_series_cleaned["description"].apply(lambda s: len(s)).max())
df_series_cleaned["description"] = df_series_cleaned["description"].apply(lambda s: s.replace("\n", ""))
df_series_cleaned["description"] = df_series_cleaned["description"].apply(lambda s: s.replace("\r", ""))
df_series_cleaned["description"] = df_series_cleaned["description"].apply(lambda s: s.replace("\t", ""))
print(sum(df_series_cleaned["description"].apply(lambda s: len(s) == 0)))
print("")

# Clean numbered column
print(set(df_series_cleaned["numbered"]))
df_series_cleaned["numbered"] = df_series_cleaned["numbered"].apply(lambda s: True if s == "true" else False)
print("")

# Rename columns
df_series_cleaned = df_series_cleaned.rename(columns={"series_id": "id"})

# Save to disk
df_series_cleaned.to_csv("series.csv", sep="\t", index=False)

### Creating `In_Series` Dataset

In [ ]:
valid_book_ids = set(df_books_cleaned["id"])
valid_series_ids = set(df_series_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)][["book_id", "series"]]

d = {"book_id": [], "series_id": []}

for i, row in df_books_.iterrows():
    for series_id in row["series"]:
        if int(series_id) in valid_series_ids:
            d["book_id"].append(row["book_id"])
            d["series_id"].append(int(series_id))

# Create dataframe
in_series = pd.DataFrame.from_dict(d)

# Drop duplicate rows
in_series = in_series.drop_duplicates()

# Print column data types
print(in_series.dtypes)

# Save to disk
in_series.to_csv("in_series.csv", sep="\t", index=False)

### Creating `Author` Dataset

In [ ]:
# Create copy of dataframe
df_authors_cleaned = df_authors.copy()

# Keep neccessary columns
df_authors_cleaned = df_authors_cleaned[["author_id", "name"]]

# Print column data types
print(df_authors_cleaned.dtypes)
print("")

# Clean author_id column
print(len(df_authors_cleaned))
print(df_authors_cleaned["author_id"].nunique())
print(df_authors_cleaned["author_id"].min())
print(df_authors_cleaned["author_id"].max())
valid_book_ids = set(df_books_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)]
valid_author_ids = set(int(d["author_id"]) for ls in df_books_["authors"] for d in ls)
df_authors_cleaned = df_authors_cleaned[df_authors_cleaned["author_id"].apply(lambda x: x in valid_author_ids)]
print("")

# Clean name column
df_authors_cleaned["name"] = df_authors_cleaned["name"].apply(lambda s: s.strip(" '"))
df_authors_cleaned["name"] = df_authors_cleaned["name"].apply(lambda s: s.strip('"'))
print(df_authors_cleaned["name"].apply(lambda s: len(s)).max())
df_authors_cleaned["name"] = df_authors_cleaned["name"].apply(lambda s: s.replace("\n", ""))
df_authors_cleaned["name"] = df_authors_cleaned["name"].apply(lambda s: s.replace("\r", ""))
df_authors_cleaned["name"] = df_authors_cleaned["name"].apply(lambda s: s.replace("\t", ""))
print(sum(df_authors_cleaned["name"].apply(lambda s: len(s) == 0)))
# Remove authors with no name
df_authors_cleaned = df_authors_cleaned[df_authors_cleaned["name"].apply(lambda s: len(s) != 0)]
print("")

# Rename columns
df_authors_cleaned = df_authors_cleaned.rename(columns={"author_id": "id"})

# Save to disk
df_authors_cleaned.to_csv("author.csv", sep="\t", index=False)

### Creating `Written_By` Dataset

In [ ]:
valid_book_ids = set(df_books_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)][["book_id", "authors"]]

d = {"book_id": [], "author_id": []}

for i, row in df_books_.iterrows():
    for d_ in row["authors"]:
        d["book_id"].append(row["book_id"])
        d["author_id"].append(int(d_["author_id"]))

# Create dataframe
written_by = pd.DataFrame.from_dict(d)

# Drop duplicate rows
written_by = written_by.drop_duplicates()

# Print column data types
print(written_by.dtypes)

# Save to disk
written_by.to_csv("written_by.csv", sep="\t", index=False)

### Creating `Series_By` Dataset

In [ ]:
valid_book_ids = set(df_books_cleaned["id"])
valid_author_ids = set(df_authors_cleaned["id"])
valid_series_ids = set(df_series_cleaned["id"])
df_books_ = df_books[df_books["book_id"].apply(lambda x: x in valid_book_ids)][["authors", "series"]]

d = {"author_id": [], "series_id": []}

for i, row in df_books_.iterrows():
    for d_ in row["authors"]:
        for series_id in row["series"]:
            if int(d_["author_id"]) in valid_author_ids and int(series_id) in valid_series_ids:
                d["author_id"].append(int(d_["author_id"]))
                d["series_id"].append(int(series_id))

# Create dataframe
series_by = pd.DataFrame.from_dict(d)

# Drop duplicate rows
series_by = series_by.drop_duplicates()

# Print column data types
print(series_by.dtypes)

# Save to disk
series_by.to_csv("series_by.csv", sep="\t", index=False)

### Creating `In_Library` Dataset

In [ ]:
# Create copy of dataframe
in_library = df_interactions.copy()

# Print column data types
print(in_library.dtypes)
print("")

# Check that (user_id, book_id) pairs are unique
in_library.groupby(["book_id", "user_id"], as_index=False).count()["review_id"].max()

# Keep neccessary columns
in_library = in_library[["user_id", "book_id", "date_added", "read_at", "started_at"]]

# Check user_id range
print(in_library["user_id"].apply(lambda s: len(s)).min())
print(in_library["user_id"].apply(lambda s: len(s)).max())
print("")

# Clean book_id column
valid_book_ids = set(df_books_cleaned["id"])
in_library = in_library[in_library["book_id"].apply(lambda x: x in valid_book_ids)]
print(in_library["book_id"].min())
print(in_library["book_id"].max())
print("")

# Clean date_added column
print(sum(in_library["date_added"].apply(lambda s: len(s) == 0)))
in_library["date_added"] = pd.to_datetime(in_library["date_added"], infer_datetime_format=True, errors="coerce")
print("")

# Clean read_at column
print(sum(in_library["read_at"].apply(lambda s: len(s) == 0)))
in_library["read_at"] = pd.to_datetime(in_library["read_at"], infer_datetime_format=True, errors="coerce")
print("")

# Clean started_at column
print(sum(in_library["started_at"].apply(lambda s: len(s) == 0)))
in_library["started_at"] = pd.to_datetime(in_library["started_at"], infer_datetime_format=True, errors="coerce")
print("")

# Save to disk
in_library.to_csv("in_library.csv", sep="\t", index=False)

### Creating `User` Dataset
We only had access to the user IDs. Hence we used the [Faker](https://faker.readthedocs.io/en/master/) library to create dummy data for the users.

In [ ]:
# Keep neccessary columns
df_users_cleaned = in_library[["user_id"]]

# Drop duplicate rows and reset index
df_users_cleaned = df_users_cleaned.drop_duplicates()
df_users_cleaned = df_users_cleaned.reset_index(drop=True)

# Create fake information
fake = Faker()
len_df = len(df_users_cleaned)
# Ensure that usernames are unique
df_users_cleaned["username"] = pd.Series(fake.unique.user_name() for _ in range(len_df))
df_users_cleaned["password"] = pd.Series(fake.password() for _ in range(len_df))
df_users_cleaned["name"] = pd.Series(fake.name() for _ in range(len_df))
df_users_cleaned["email"] = pd.Series(fake.email() for _ in range(len_df))

# Rename columns
df_users_cleaned = df_users_cleaned.rename(columns={"user_id": "id"})

# Save to disk
df_users_cleaned.to_csv("user.csv", sep="\t", index=False)

### Creating `Review` Dataset

In [ ]:
# Create copy of dataframe
df_reviews_cleaned = df_reviews.copy()

# Keep neccessary columns
df_reviews_cleaned = df_reviews_cleaned[["review_id", "user_id", "book_id", "review_text", "rating", "n_votes", "n_comments"]]

# Print column data types
print(df_reviews_cleaned.dtypes)
print("")

# Check review_id column range and no. of unique values
print(len(df_reviews_cleaned))
print(df_reviews_cleaned["review_id"].nunique())
print(df_reviews_cleaned["review_id"].apply(lambda s: len(s)).min())
print(df_reviews_cleaned["review_id"].apply(lambda s: len(s)).max())
print("")

# Clean book_id column
valid_book_ids = set(df_books_cleaned["id"])
df_reviews_cleaned = df_reviews_cleaned[df_reviews_cleaned["book_id"].apply(lambda x: x in valid_book_ids)]
print(df_reviews_cleaned["book_id"].min())
print(df_reviews_cleaned["book_id"].max())
print("")

# Clean user_id column
valid_user_ids = set(df_users_cleaned["id"])
df_reviews_cleaned = df_reviews_cleaned[df_reviews_cleaned["user_id"].apply(lambda x: x in valid_user_ids)]
print(df_reviews_cleaned["user_id"].apply(lambda s: len(s)).min())
print(df_reviews_cleaned["user_id"].apply(lambda s: len(s)).max())
print("")

# Clean review_text column
df_reviews_cleaned["review_text"] = df_reviews_cleaned["review_text"].apply(lambda s: s.strip(" '"))
df_reviews_cleaned["review_text"] = df_reviews_cleaned["review_text"].apply(lambda s: s.strip('"'))
print(df_reviews_cleaned["review_text"].apply(lambda s: len(s)).max())
df_reviews_cleaned["review_text"] = df_reviews_cleaned["review_text"].apply(lambda s: s.replace("\n", ""))
df_reviews_cleaned["review_text"] = df_reviews_cleaned["review_text"].apply(lambda s: s.replace("\r", ""))
df_reviews_cleaned["review_text"] = df_reviews_cleaned["review_text"].apply(lambda s: s.replace("\t", ""))
print(sum(df_reviews_cleaned["review_text"].apply(lambda s: len(s) == 0)))
print("")

# Clean rating column
print(sum(df_reviews_cleaned["rating"].apply(lambda x: x < 0 or x > 5)))
print(sum(df_reviews_cleaned["rating"].apply(lambda x: x == 0)))
df_reviews_cleaned["rating"] = df_reviews_cleaned["rating"].apply(lambda x: x if x != 0 else np.nan)
print("")

# Clean n_votes column
print(sum(df_reviews_cleaned["n_votes"].apply(lambda x: x < 0)))
df_reviews_cleaned["n_votes"] = df_reviews_cleaned["n_votes"].apply(lambda x: x if x >= 0 else 0)
print("")

# Clean n_comments column
print(sum(df_reviews_cleaned["n_comments"].apply(lambda x: x < 0)))
df_reviews_cleaned["n_comments"] = df_reviews_cleaned["n_comments"].apply(lambda x: x if x >= 0 else 0)
print("")

# Rename columns
df_reviews_cleaned = df_reviews_cleaned.rename(columns={"review_id": "id", "review_text": "text", "n_votes": "num_votes", "n_comments": "num_comments"})

# Save to disk
df_reviews_cleaned.to_csv("review.csv", sep="\t", index=False)